In [ ]:
# !pip install autogluon

NaN Verileri Doldurunca Başarım Arttı: 0.80383

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
from autogluon.tabular import TabularPredictor

In [ ]:
test_data = pd.read_csv("/content/drive/MyDrive/spaceship/kaggle/test.csv")

In [ ]:
train_data = pd.read_csv("/content/drive/MyDrive/spaceship/kaggle/train.csv")

EDA

In [ ]:
def countDestinationValues(df): 
  destination_list = df["Destination"].tolist()
  destinations = df["Destination"].unique().tolist()
  # destinations.remove(np.nan)
  nan_count = destination_list.count(np.nan)
  destination_length = len(destination_list) - nan_count
  print("Without nan:",destination_length)
  for destination in destinations:
    rate = (destination_list.count(destination) / destination_length) * 100
    print(f"{destination}, {destination_list.count(destination)}, rate: % {rate}")

def countVIP(df):
  values = df["VIP"].unique().tolist()
  vip_list = df["VIP"].tolist()
  for value in values:
    print(value,vip_list.count(value))

def countCabinValues(df):
  cabin_1_values = df["Cabin_1"].unique().tolist()
  cabin_1 = df["Cabin_1"].tolist()
  for value in cabin_1_values:
    print(value,cabin_1.count(value))

  cabin_2_values = df["Cabin_2"].unique().tolist() 
  cabin_2 = df["Cabin_2"].tolist()
  # print(max(cabin_2))
  for value in cabin_2_values:
    print(value,cabin_2.count(value))

  cabin_3_values = df["Cabin_3"].unique().tolist()
  cabin_3 = df["Cabin_3"].tolist()
  for value in cabin_3_values:
    print(value,cabin_3.count(value))

def countHomePlanet(df):
  values = df["HomePlanet"].unique().tolist()
  homeplanet_list = df["HomePlanet"].tolist()
  for value in values:
    print(value,homeplanet_list.count(value))

In [ ]:
def giveGeneralInfo(df):
  print("Columns: ", df.columns.tolist())
  print("-------------------")
  print(df.info())
  print("-----------------")
  print(df.isna().sum())

def splitPassengerId(df):
  df[["PassengerId_1","PassengerId_2"]] = df["PassengerId"].str.split("_", expand = True)
  df = df.drop("PassengerId",axis = 1)
  return df

def splitCabin(df):
  df["Cabin"] = df["Cabin"].fillna("F/0/S")
  df[["Cabin_1","Cabin_2","Cabin_3"]] = df["Cabin"].str.split("/",expand=True)
  df = df.drop("Cabin",axis = 1)
  return df

def calculateTotalAmount(df):
  df["TotalAmount"] = df.iloc[:,7:12].sum(axis = 1)
  return df

def fillNanValuesInVip(df): # Başarı oranını düşürdü.
  value = False
  vip_list = df["VIP"].unique()
  if value in vip_list:
    df["VIP"] = df["VIP"].fillna(value)
  return df

def fillNanValuesInDestination(df): # Destination'daki NaN verileri oransal olarak en çok olan değer ile doldurdum.
  destination_list = df["Destination"].unique()
  value = "TRAPPIST-1e"
  if value in destination_list:
    df["Destination"] = df["Destination"].fillna(value)
  return df

def setAgeGroup(df):
  df["ageGroup"] = "Adult"
  df.loc[df["Age"] < 2, "ageGroup"] = "Baby"
  df.loc[(2 < df["Age"]) & (df["Age"] < 12), "ageGroup"] = "Kid"
  df.loc[df["Age"] > 65, "ageGroup"] = "Old"
  # df = df.drop("Age",axis=1) # Age column'unu kaldırınca başarı oranı düşüyor.
  return df

def dropNameColumn(df): # duruma göre kaldırılabilir.
  df = df.drop("Name",axis = 1)
  return df

def dropSomeColumns(df):
  df = df.drop(["RoomService","FoodCourt","ShoppingMall","Spa","VRDeck"],axis = 1)
  return df

def fillAgeWithAverageAge(df):
  sum = df["Age"].sum(axis=0)
  length = len(df["Age"])
  average_age = sum/length
  average_age = float(round(average_age))
  df["Age"] = df["Age"].fillna(average_age)
  return df

def createRoomServiceBool(df):
  df["RoomServiceBool"] = False
  df.loc[df["RoomService"] > 0, "RoomServiceBool"] = True
  return df

def createFoodCourtBool(df):
  df["FoodCourtBool"] = False
  df.loc[df["FoodCourt"] > 0, "FoodCourtBool"] = True
  return df

def createShoppingMallBool(df):
  df["ShoppingMallBool"] = False
  df.loc[df["ShoppingMall"] > 0, "ShoppingMallBool"] = True
  return df

def createSpaBool(df):
  df["SpaBool"] = False
  df.loc[df["Spa"] > 0, "SpaBool"] = True
  return df

def fillNanValuesInHomePlanet(df):
  value = "Earth"
  homeplanet_list = df["HomePlanet"].unique().tolist()
  if value in homeplanet_list:
    df["HomePlanet"] = df["HomePlanet"].fillna(value)
  return df

def fillNanValuesInCryoSleep(df):
  value = False
  cryosleep_list = df["CryoSleep"].unique().tolist()
  if value in cryosleep_list:
    df["CryoSleep"] = df["CryoSleep"].fillna(value)
  return df

def dropNanRows(df):
  nan_rows = df.index[df.isnull().any(axis = 1)]
  df = df.drop(nan_rows,axis = 0)
  return df

def allProcess(df):
  df = calculateTotalAmount(df)
  df = splitPassengerId(df)
  df = splitCabin(df)
  df = fillNanValuesInDestination(df)
  df = setAgeGroup(df)
  df = fillNanValuesInVip(df)
  df = fillNanValuesInHomePlanet(df)
  df = fillNanValuesInCryoSleep(df)
  df = fillAgeWithAverageAge(df)
  df = createRoomServiceBool(df)
  df = createFoodCourtBool(df)
  df = createShoppingMallBool(df)
  df = createSpaBool(df)
  # df = dropNanRows(df) 
  # df = dropSomeColumns(df)
  return df

def predict(train_data,test_data):
  label = 'Transported'
  predictor = TabularPredictor(label=label).fit(train_data)
  y_pred = predictor.predict(test_data)
  return y_pred

def createSubmissionCsv(train_data,test_data):
  submission_df = pd.DataFrame()
  submission_df["PassengerId"] = test_data["PassengerId_1"].astype(str) + "_" +test_data["PassengerId_2"].astype(str)
  submission_df["Transported"] = predict(train_data,test_data)
  submission_df = submission_df.set_index("PassengerId")
  return submission_df

In [ ]:
train_data = allProcess(train_data)

In [ ]:
test_data = allProcess(test_data)

In [ ]:
giveGeneralInfo(train_data)

In [ ]:
submission_df = createSubmissionCsv(train_data,test_data)

In [ ]:
submission_df.to_csv("/content/drive/MyDrive/spaceship/20th_pred.csv")